# Домашнє завдання 3

## 1. Завантаження векторів слів

In [38]:
import pickle
import pandas as pd
import numpy as np
from sklearn.decomposition import PCA

file_name = 'word_embeddings_subset.p'

with open(file_name, 'rb') as f:
    word_embeddings = pickle.load(f)

### Огляд завантажених даних
print("1. Огляд завантажених даних:")
print("-" * 30)
print(f"Довжина завантаженого набору даних з моделі: {len(word_embeddings)} слів.\n")

country_vector = word_embeddings['country']
print(f"Приклад даних: \nВектор для слова 'country' (перші 10 з {len(country_vector)} координат):")
print(country_vector[:10])
print("Розмірність вектора", np.shape(country_vector), "\n")

words = list(word_embeddings.keys())
print(f"Список слів у моделі (перші 25 з {len(words)} слів):")
print(words[:25])

vectors = np.array(list(word_embeddings.values()))
print(f"Розмірність початкової матриці векторів: {vectors.shape}\n")

### Відбір трьох головних компонент
print("\n2. Відбір трьох головних компонент:")
print("-" * 30)

# Навчаємо модель PCA на наших даних і трансформуємо їх
# Результатом буде нова матриця з векторами розмірності 3
pca = PCA(n_components=3)
reduced_vectors = pca.fit_transform(vectors)
print(f"Розмірність матриці після PCA: {reduced_vectors.shape}\n")

### Датафрейм з новими даними
print("\n3. Датафрейм з новими даними:")
print("-" * 30)

# Тепер 'x', 'y', 'z' - це значення головних компонент
df_pca = pd.DataFrame(data=reduced_vectors, index=words, columns=['x', 'y', 'z'] )

print("DataFrame, створений за допомогою PCA (перші 10 рядків):")
print(df_pca.head(10))

1. Огляд завантажених даних:
------------------------------
Довжина завантаженого набору даних з моделі: 243 слів.

Приклад даних: 
Вектор для слова 'country' (перші 10 з 300 координат):
[-0.08007812  0.13378906  0.14355469  0.09472656 -0.04736328 -0.02355957
 -0.00854492 -0.18652344  0.04589844 -0.08154297]
Розмірність вектора (300,) 

Список слів у моделі (перші 25 з 243 слів):
['country', 'city', 'China', 'Iraq', 'oil', 'town', 'Canada', 'London', 'England', 'Australia', 'Japan', 'Pakistan', 'Iran', 'gas', 'happy', 'Russia', 'Afghanistan', 'France', 'Germany', 'Georgia', 'Baghdad', 'village', 'Spain', 'Italy', 'Beijing']
Розмірність початкової матриці векторів: (243, 300)


2. Відбір трьох головних компонент:
------------------------------
Розмірність матриці після PCA: (243, 3)


3. Датафрейм з новими даними:
------------------------------
DataFrame, створений за допомогою PCA (перші 10 рядків):
                  x         y         z
country    0.746037 -0.387964 -0.482691
city   

## 2. Функція для пошуку найближчого слова

In [39]:
from numpy.linalg import norm

def find_closest_word_cosine(input_vector, df):
    """
    Знаходить найбільш подібне слово (з ІНДЕКСУ DataFrame) до заданого вектора,
    використовуючи косинусну подібність.
    """
    embedding_vectors = df[['x', 'y', 'z']].values
    input_vec_np = np.array(input_vector)
    
    dot_products = np.dot(embedding_vectors, input_vec_np)
    norms = norm(embedding_vectors, axis=1) * norm(input_vec_np)
    
    # Уникаємо ділення на нуль, якщо вхідний вектор нульовий
    if norms.all() == 0:
        return None
    
    similarities = dot_products / norms
    
    closest_index = np.argmax(similarities)
    closest_word = df.index[closest_index]
    
    return closest_word

In [40]:
import random
import plotly.express as px

# Вибираємо випадкову вибірку з 25 слів для візуалізації
words_sample = random.sample(words, 25)
df_sample = df_pca.loc[words_sample]

# Візуалізація (необов'язково)

fig = px.scatter_3d(
    df_sample,
    x='x',
    y='y',
    z='z',
    text=df_sample.index,  # показати назви країн/слів
    title='3D PCA Visualization'
)

fig.update_traces(marker=dict(size=3), textposition='top center')
fig.show()

In [41]:
### Перевірка довільних векторів координат
test_vectors = [
    [0, 0, 0],
    [0.92, 0.4, 0.365],
    [-0.5, 0.5, -0.5],
    [2, 0, 0],
    [-2, 0, 0],
    [0, 2, 0],
    [0, -2, 0]
]

print("--- Перевірка довільних векторів координат ---")
print("-" * 50)

for i, vector in enumerate(test_vectors):
    # Знаходимо найближче слово для поточного вектора
    # Пошук ведемо по всьому відібраному DataFrame df_sample
    closest_word = find_closest_word_cosine(vector, df_sample)

    vector_str = f"[{vector[0]:.2f}, {vector[1]:.2f}, {vector[2]:.2f}]"
    print(f"Для вектора {vector_str:<20} найближчим є слово: '{closest_word}'")

print("-" * 50)

--- Перевірка довільних векторів координат ---
--------------------------------------------------
Для вектора [0.00, 0.00, 0.00]   найближчим є слово: 'None'
Для вектора [0.92, 0.40, 0.36]   найближчим є слово: 'Cyprus'
Для вектора [-0.50, 0.50, -0.50] найближчим є слово: 'Kathmandu'
Для вектора [2.00, 0.00, 0.00]   найближчим є слово: 'Denmark'
Для вектора [-2.00, 0.00, 0.00]  найближчим є слово: 'Tunis'
Для вектора [0.00, 2.00, 0.00]   найближчим є слово: 'Belgrade'
Для вектора [0.00, -2.00, 0.00]  найближчим є слово: 'Senegal'
--------------------------------------------------


## 3. Векторний добуток

In [44]:
### Генерація випадкових пар слів
print(f"Загальна кількість слів для аналізу: {len(words)}\n")

proper_nouns = []
common_words = []

for word in words:
    # Перевіряємо, чи слово не порожнє і чи починається з великої літери
    if word and word[0].isupper():
        proper_nouns.append(word)
    else:
        common_words.append(word)

print(f"Знайдено {len(proper_nouns)} власних назв (напр., країни, міста).")
print(f"Знайдено {len(common_words)} загальних слів.\n")
print("-" * 50)

random_pairs = []

# Генеруємо 10 пар
for _ in range(10):
    # Вибираємо по одному випадковому слову з кожного списку
    random_proper = random.choice(proper_nouns)
    random_common = random.choice(common_words)
    random_pairs.append((random_proper, random_common))

# Виводимо результат
print("10 випадково згенерованих пар (власна назва, загальне слово):")
for pair in random_pairs:
    print(f"  - {pair[0]}, {pair[1]}")

Загальна кількість слів для аналізу: 243

Знайдено 230 власних назв (напр., країни, міста).
Знайдено 13 загальних слів.

--------------------------------------------------
10 випадково згенерованих пар (власна назва, загальне слово):
  - Kabul, king
  - Rome, sad
  - Bangkok, oil
  - Eritrea, gas
  - Athens, gas
  - Managua, queen
  - England, sad
  - Vienna, oil
  - Belarus, continent
  - Georgia, queen


In [45]:
print("\n--- Обчислення векторного добутку для пошуку ортогонального слова ---")
print("-" * 60)

for pair in random_pairs:
    word1, word2 = pair

    if word1 in df_pca.index and word2 in df_pca.index:
        # Отримуємо вектори
        vec1 = df_pca.loc[word1].values
        vec2 = df_pca.loc[word2].values
        
        # Обчислюємо векторний добуток
        cross_vector = np.cross(vec1, vec2)
        
        # Знаходимо найближче слово до результату
        orthogonal_word = find_closest_word_cosine(cross_vector, df_pca)
        
        print(f"Векторний добуток '{word1}' та '{word2}' -> знаходить слово: '{orthogonal_word}'\n")
    else:
        print(f"Пара '{word1}'-'{word2}' не може бути оброблена: одне зі слів відсутнє в даних.\n")

print("-" * 60)


--- Обчислення векторного добутку для пошуку ортогонального слова ---
------------------------------------------------------------
Векторний добуток 'Kabul' та 'king' -> знаходить слово: 'Liberia'

Векторний добуток 'Rome' та 'sad' -> знаходить слово: 'Monrovia'

Векторний добуток 'Bangkok' та 'oil' -> знаходить слово: 'Liberia'

Векторний добуток 'Eritrea' та 'gas' -> знаходить слово: 'Armenia'

Векторний добуток 'Athens' та 'gas' -> знаходить слово: 'Belmopan'

Векторний добуток 'Managua' та 'queen' -> знаходить слово: 'Liberia'

Векторний добуток 'England' та 'sad' -> знаходить слово: 'Stockholm'

Векторний добуток 'Vienna' та 'oil' -> знаходить слово: 'Belmopan'

Векторний добуток 'Belarus' та 'continent' -> знаходить слово: 'gas'

Векторний добуток 'Georgia' та 'queen' -> знаходить слово: 'Damascus'

------------------------------------------------------------


Висновки.

Проведені експерименти демонструють, що векторна арифметика є потужним інструментом для дослідження семантичних зв'язків між словами, "закодованих" у моделі word embeddings. Навіть якщо арифметичні операції з векторами не демонструють інтуїтивно зрозуміли результати, все одно це - потужний інструмент для досліджень складних, прихованих зв'язків. 

Операція пошуку ортогонального слова є найбільш абстрактною, так як показує напрямок максимально незалежний до семантичної площини, яку створюють два вектори, наприклад, ('Georgia', 'queen') та 'Damascus'. Ортогональний вектор не вказує на якусь легко вгадувану "третю" концепцію.

Найбільш разючий висновок полягає в тому, що майже всі результати є власними назвами (країнами або містами). Навіть коли ми поєднуємо країну з емоцією ('Rome' та 'sad'), результат все одно є географічним об'єктом ('Monrovia'). Коли ми обчислюємо векторний добуток, ми знаходимо новий напрямок, перпендикулярний до площини перших двох слів. Але оскільки весь простір "заселений" переважно країнами та містами, найближчою "точкою" до кінця цього нового вектора майже завжди виявляється інша країна чи місто.


## 4. Кут між словами

In [47]:
def angle_between_words(word1, word2, df):
    """
    Обчислює кут (у градусах) між векторами двох слів.

    Аргументи:
    word1 (str): Перше слово.
    word2 (str): Друге слово.
    df (pd.DataFrame): DataFrame з векторами слів (індекс - слова).

    Повертає:
    float: Кут між векторами у градусах, або None, якщо одне зі слів відсутнє.
    """
    # Перевіряємо, чи обидва слова є в DataFrame
    if word1 not in df.index or word2 not in df.index:
        return None

    vec1 = df.loc[word1].values
    vec2 = df.loc[word2].values

    # Обчислюємо косинусну подібність (cos θ)
    cosine_similarity = np.dot(vec1, vec2) / (norm(vec1) * norm(vec2))

    # np.clip гарантує, що значення буде в межах [-1, 1] для уникнення помилок обчислень
    angle_in_radians = np.arccos(np.clip(cosine_similarity, -1.0, 1.0))
    angle_in_degrees = np.degrees(angle_in_radians)

    return angle_in_degrees

print("--- Обчислення кута між словами ---")
print("-" * 50)

num_pairs = 10
random_word_pairs = []
for _ in range(num_pairs):
    word1, word2 = random.sample(words, 2)
    random_word_pairs.append((word1, word2))

for word1, word2 in random_word_pairs:
    angle = angle_between_words(word1, word2, df_pca)
    if angle is not None:
        print(f"Кут між '{word1}' та '{word2}': {angle:.2f}°")
    else:
        print(f"Не вдалося обчислити кут для '{word1}' та '{word2}' (слово відсутнє).")

print("-" * 50)

--- Обчислення кута між словами ---
--------------------------------------------------
Кут між 'Skopje' та 'Vienna': 44.73°
Кут між 'Burundi' та 'Tbilisi': 108.18°
Кут між 'Bujumbura' та 'Rabat': 41.45°
Кут між 'village' та 'Bamako': 123.67°
Кут між 'Baghdad' та 'Honduras': 142.80°
Кут між 'Fiji' та 'Eritrea': 66.03°
Кут між 'oil' та 'Zimbabwe': 83.61°
Кут між 'Tirana' та 'Mauritania': 121.52°
Кут між 'Monrovia' та 'Skopje': 121.56°
Кут між 'London' та 'Dominica': 89.78°
--------------------------------------------------


Висновки.

Аналіз кутів підтверджує, що модель здатна відтворювати очевидні зв'язки, але також можливо виявляти приховані подібності, наприклад соціальні.

1) Схожі та пов'язані поняття (Малий кут < 45°)

"Африканська столиця": 'Bujumbura' та 'Rabat' (41.45°).

"Європейські столиці": 'Skopje' та 'Vienna' (44.73°).

2) Семантично незалежні слова (Кут ≈ 90°)

'London' та 'Dominica' (89.78°): Глобальний мегаполіс та мала острівна держава.

'oil' та 'Zimbabwe' (83.61°): Зімбабве не є великим виробником нафти.

3) Протижлежні поняття (Великий кут > 100°)

'village' та 'Bamako' (123.67°): Загальне поняття "село" протилежне конкретній великій столиці "Бамако".

'Baghdad' та 'Honduras' (142.80°): Аналогічно, максимальна віддаленість.
